In [1]:
import numpy as np
import pandas as pd

# Входные данные

In [2]:
shell = 1 # рассчитываем оболочку или нет
R = 1.5 # радиус оболочки (np.inf - прямая пластина)
plate = 1
plate1 = 0

In [18]:
E = 7e10  # модуль Юнга
ro = 2700  # плотоность алюминия
nu = 0.33  # коэффицент Пуассона
G = E / (2 * (1 + nu))  # модуль сдвига
L = 1.5  # L1  
W = 1.5   # L2
h = 0.0015   # тодщина пластины
D = (E * h ** 3) / (12 * (1 - nu ** 2))  # жесткость из уравнения колебания
N_end = 30  # расчтеные моды
M_end = 30
q1_end = 0   # число шпангаутов
q2_end = 0   # число стрингеров
h_sh = 0.1 # высота шпангоута
h_st = 0.02 # высота стрингера
b_sh_st = 0.002 # толщина сечения стрингера и шпангоута
S = N_end * M_end

In [19]:
if shell == 0 and R < 10000:
    plate1 = 1
    plate = 0
elif shell == 1:
    W = 2 * np.pi * R
    plate = 0
    plate1 = 0

# Расчет характеристик стрингеров и шпангоутов

In [20]:
S_sh = h_sh * b_sh_st  # профиль шпангоута
S_st = h_st * b_sh_st # профиль стрингера
I_sh = b_sh_st * (h_sh ** 3) / 12  # моменты инерции относительно оси y (толщина сечения)
I_st = b_sh_st * (h_st ** 3) / 12
# моменты кручения
Ik_sh = ((b_sh_st ** 3) * h_sh / 3) * (1 - 0.63 * (b_sh_st / h_sh) + 0.052 * (b_sh_st / h_sh) ** 5)
Ik_st = ((b_sh_st ** 3) * h_st / 3) * (1 - 0.63 * (b_sh_st / h_st) + 0.052 * (b_sh_st / h_st) ** 5)
# полярные моменты
Iro_sh = 0.5 * (b_sh_st * h_sh) * (h_sh ** 2 + b_sh_st ** 2)
Iro_st = 0.5 * (b_sh_st * h_st) * (h_st ** 2 + b_sh_st ** 2)
# погонные плотности шпангаутов и стрингеров (на единицу длины пластины)
ma_sh = ro * S_sh
ma_st = ro * S_st

In [21]:
# геометрия стрингеров и шпангоутов
b_pan_sh = L / (q1_end + 1) # пролет шпангаута
b_pan_st = W / (q2_end + 1) # пролет стрингера
S_pan_sh = b_pan_sh * h
S_pan_st = b_pan_st * h
V_sh = S_pan_sh * (h / 2 + h_sh / 2) / (S_pan_sh + S_sh)
V_st = S_pan_st * (h / 2 + h_st / 2) / (S_pan_st + S_st)

In [22]:
# координаты стрингеров и шпангоутов
x1 = np.arange(1, q1_end + 1) * b_pan_sh
x2 = np.arange(1, q2_end + 1) * b_pan_st

# Задаем пластину

In [23]:
# функции для вычисления ki, kj
def ku1(i):
    return (np.pi * i) / L
def ku2(j):
    return plate * (np.pi * j) / W + plate1 * (np.pi * j) / W + shell * j / R

In [24]:
# задаем матрицу оператора d^4/dx^4
Z0_eig = np.zeros((S, S))
for i in range(1, M_end + 1):
    for j in range(1 ,N_end + 1):
        k = (i - 1) * N_end + j
        ku = ku1(i) ** 4 + 2 * (ku1(i) * ku2(j)) ** 2 + ku2(j) ** 4
        Z0_eig[k-1][k-1] = D * ku + (plate1 * E * h * ku1(i) ** 4) / (ku * R ** 2) +(shell * E * h * ku1(i) ** 4) / (ku * R ** 2)

In [25]:
# задаем массовую матрицу
Z0_eig_m = np.diag([ro * h] * S)

In [26]:
# матрицы интегралов
V1 = np.diag([L / 2] * M_end)
V2 = np.diag([W / 2] * N_end)

# Матрицы для шпангоутов

In [27]:
sh10 = np.zeros((S, S))
sh20 = np.zeros((S, S))
for m1 in range(1, M_end + 1):
    for n1 in range(1, N_end + 1):
        for m2 in range(1, M_end + 1):
            for n2 in range(1, N_end + 1):
                j1 = (m1 - 1) * N_end + n1
                j2 = (m2 - 1) * N_end + n2
                Sum_r10_1, Sum_r20_1, Sum_r10_2, Sum_r20_2 = [0] * 4
                sh10_0 = E * (I_sh + S_sh * V_sh ** 2) * ku2(n1) ** 4
                sh20_0 = G * Ik_sh * (ku1(m1) * ku2(n1)) ** 2
                for q1 in x1:
                    # упругая матрица
                    Sum_r10_1 += (4 / (L * W)) * sh10_0 * np.sin(ku1(m1) * q1) * np.sin(ku1(m2) * q1) * V2[n1-1][n2-1]
                    Sum_r10_2 += (4 / (L * W)) * sh20_0 * np.sin(ku1(m1) * q1) * np.sin(ku1(m2) * q1) * V2[n1-1][n2-1]
                    # массовая матрица
                    Sum_r20_1 += (4 / (L * W)) * ma_sh * np.sin(ku1(m1) * q1) * np.sin(ku1(m2) * q1) * V2[n1-1][n2-1]
                    Sum_r20_2 += (4 / (L * W)) * ro * Iro_sh * (ku1(m1) ** 2) * np.sin(ku1(m1) * q1) * np.sin(ku1(m2) * q1) * V2[n1-1][n2-1]
                    
                sh10[j1-1][j2-1] = Sum_r10_1 + Sum_r10_2  # упругая
                sh20[j1-1][j2-1] = Sum_r20_1 + Sum_r20_2  # массовая

# Матрицы для стрингеров

In [28]:
st10 = np.zeros((S, S))
st20 = np.zeros((S, S))
for m1 in range(1, M_end + 1):
    for m2 in range(1, M_end + 1):
        for n1 in range(1, N_end + 1):
            for n2 in range(1, N_end + 1):
                j1 = (m1 - 1) * N_end + n1
                j2 = (m2 - 1) * N_end + n2
                Sum_r10_1, Sum_r20_1, Sum_r10_2, Sum_r20_2 = [0] * 4
                st10_0 = E * (I_st + S_st * V_st ** 2) * ku1(m1) ** 4
                st20_0 = G * Ik_st * (ku1(m1) * ku2(n1)) ** 2
                for q2 in x2:
                    # упругая матрица
                    slag1 = plate * np.sin(ku2(n1) * q2) * np.sin(ku2(n2) * q2) + shell * np.cos(ku2(n1) * q2) * np.cos(ku2(n2) * q2)
                    Sum_r10_1 += (4 / (L * W)) * st10_0 * V1[m1-1][m2-1] * slag1
                    slag2 = plate * np.cos(ku2(n1) * q2) * np.cos(ku2(n2) * q2) + shell * np.sin(ku2(n1) * q2) * np.sin(ku2(n2) * q2)
                    Sum_r10_2 += (4 / (L * W)) * st20_0 * ku2(n1) * ku2(n2) * V1[m1-1][m2-1] * slag2
                    # массовая матрица
                    Sum_r20_1 += (4 / (L * W)) * ma_st * V1[m1-1][m2-1] * slag1
                    Sum_r20_2 += (4 / (L * W)) * ro * Iro_st * V1[m1-1][m2-1] * slag2
                    
                st10[j1-1][j2-1] = Sum_r10_1 + Sum_r10_2  # упругая
                st20[j1-1][j2-1] = Sum_r20_1 + Sum_r20_2  # массовая

In [29]:
# упругая суммарная
Z0_eig_1 = Z0_eig + sh10 + st10
# массовая суммарная
Z0_eig_2 = Z0_eig_m + sh20 + st20

In [30]:
# финальная матрица
Z0_eig_final = np.linalg.inv(Z0_eig_2).T @ Z0_eig_1

In [31]:
# считаем собственные частоты
A, B = np.linalg.eig(Z0_eig_final)
eig_f = np.sqrt(np.sort(A)).reshape(N_end, M_end)
pd.DataFrame(eig_f / (np.pi * 2))

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,41.975143,42.751019,42.977625,44.956541,46.124843,48.286734,51.814424,52.501171,57.424820,60.568306,...,89.576036,90.949274,91.094925,91.719776,95.601689,97.219751,98.575885,101.136055,105.223932,107.427141
1,107.946927,113.580409,114.376157,116.490817,120.131392,121.906647,122.283414,125.959744,126.390715,127.121026,...,141.079344,141.522380,147.306410,147.468896,150.376565,150.435764,153.023002,154.327504,154.641056,156.864799
2,161.465361,162.056548,167.972048,168.238710,169.129549,170.031122,170.422315,171.602829,173.440648,174.931729,...,206.157585,206.821065,209.671883,210.023589,210.060044,211.307183,211.487809,213.832292,214.368312,217.161229
3,217.516129,218.751385,222.274739,224.674459,231.318564,231.607102,232.162595,241.228988,241.499057,248.046283,...,264.091852,266.495682,267.458667,274.619743,277.855145,282.591692,282.849510,283.124257,283.964018,286.943100
4,292.946019,293.125498,293.839096,294.209412,294.613170,295.749909,296.497920,298.477220,302.397867,304.045772,...,330.953111,333.514823,335.871383,335.994206,336.800108,337.154404,338.786608,339.355715,341.828400,344.395457
5,345.913718,347.205932,348.925318,349.946479,351.021118,357.119226,359.262483,359.897748,361.455477,364.166575,...,383.817662,384.449585,384.552003,384.995500,386.186998,386.849136,387.605767,388.687386,390.434977,392.262531
6,393.193420,397.748560,400.503466,400.657375,404.016014,410.318323,410.617676,411.008624,411.464295,413.727241,...,426.902448,427.857755,428.252762,429.014232,431.788057,432.042391,434.395099,435.649608,436.414097,440.026938
7,441.118943,441.687155,443.526302,444.814080,445.820058,446.271380,447.551853,453.379869,453.946356,454.298930,...,463.999796,464.875169,467.298019,467.389810,468.046714,470.140886,470.351506,472.054284,473.772230,473.805423
8,473.994615,475.599178,478.181010,479.451296,481.821102,482.859242,483.375508,483.598433,485.613691,487.973012,...,498.378179,499.239957,499.259715,502.653038,503.695553,503.832272,503.950964,504.074342,504.861295,505.062209
9,505.299006,506.442934,506.629611,507.139488,508.632917,508.742636,509.205566,510.097029,511.363112,511.507505,...,520.598321,521.827590,522.385464,522.888070,522.948652,524.132299,526.018060,526.947317,527.583473,527.996659


In [54]:
idx = A.argsort()   
A = A[idx]
B = B[:,idx]

# Рисуем решение

In [55]:
x = np.linspace(0, L, 1000)
y = np.linspace(0, W, 1000)
xx, yy = np.meshgrid(x, y)

In [61]:
mode = 2
w = np.sqrt(A[mode - 1]) / (np.pi * 2)

In [62]:
b = B[:, mode-1] # eigen vector

In [63]:
s = lambda i, j: (i - 1) * N_end + j - 1
zz1 = plate * np.array([[b[s(i, j)] * np.sin(ku1(i) * xx) * np.sin(ku2(j) * yy) for i in range(1, M_end + 1)] for j in range(1, N_end + 1)])
zz2 = shell * np.array([[b[s(i, j)] * np.sin(ku1(i) * xx) * np.cos(j * yy / R) for i in range(1, M_end + 1)] for j in range(1, N_end + 1)])

zz = zz1 + zz2

In [64]:
zz = np.sum(zz, (0, 1))

In [65]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
%matplotlib notebook
# Рисуем моду
fig = plt.figure()
ax = fig.add_subplot(111)
ax.contour(xx, yy, zz * (-1), cmap=cm.coolwarm)

<IPython.core.display.Javascript object>

In [29]:
# visualisation
fps = 10 # frame per sec
frn = 50 # frame number of the animation

In [32]:
t = np.linspace(0, 4 * np.pi / w, frn)
z_arr = np.array([zz * np.cos(w * i + np.pi / 2) for i in t])

In [33]:
def update_plot(frame_number, zarray, plot):
    plot[0].remove()
    plot[0] = ax.plot_surface(xx, yy, zarray[frame_number], cmap="magma")

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

plot = [ax.plot_surface(xx, yy, z_arr[0], color='0.75', rstride=1, cstride=1)]
ax.set_zlim(0,1.1)
ani = animation.FuncAnimation(fig, update_plot, frn, fargs=(z_arr, plot), interval=1000/fps)

<IPython.core.display.Javascript object>

NameError: name 'animation' is not defined

In [39]:
fn = '3x4mode'
ani.save(fn+'.gif',writer='imagemagick',fps=fps)